# Functional Closures and Decorators

##  Closures
* In order to understand closures, let's review the Python scoping rules: LEGB
  * L = local
  * E = enclosing
  * G = global
  * B = builtin (e.g., len() function)

In [1]:
global_var = 'global var'
   
def outer_func():
    local_var = 'local to outer_func()'
    def inner_func():
        inner_var = 'local to inner_func()'
        print(local_var, 'enclosing scope')
        print(global_var, 'global scope')
        
    inner_func()

In [ ]:
outer_func()

local to outer_func() enclosing scope
global var global scope


* When a function references a name that is not local, Python first attempts to resolve that name in the enclosing scope
* A *closure* is a nested function which _remembers_ a value or values from the enclosing lexical scope even when the program flow is no longer in the enclosing scope

In [6]:
def make_adder(x):
    # print x's address
    print(f'id(x): 0x{id(x):x}')
    # enclosing scope
    
    def adder(y):
        print('in adder')
        return x + y # Python uses LEGB to find 'x'
    
    print(f'id(adder): 0x{id(adder):x}')
    return adder # we return the function adder (i.e., its address in memory) 

add39 = make_adder(39)
print('about to call add39()')
add39(109)

id(x): 0x7fb80802edf0
id(adder): 0x7fb7e9791040
about to call add39()
in adder


148

In [7]:
# let's use repr so we can see the address of the function
# we could use print("%X") as well...
print(type(add39), repr(add39), sep='\n')

<class 'function'>
<function make_adder.<locals>.adder at 0x7fb7e9791040>


In [8]:
# all functions have a closure attribute
add39.__closure__

(<cell at 0x7fb7e97ace20: int object at 0x7fb80802edf0>,)

In [9]:
# notice that the cell object has a reference to an int object
add39.__closure__[0].cell_contents

39

In [10]:
print(make_adder.__closure__)

None


* One case where closures are frequently used is in building function wrappers
* Suppose we want to log each invocation of a function:

In [16]:
def logging(f):
    def wrapper(*args, **kwargs):
        print(f'Calling {f}({args}, {kwargs})')
        return f(*args, **kwargs)
    return wrapper

In [17]:
logging_add39 = logging(add39)

In [18]:
print(add39(5)) # remember that add39 just adds 39 to our argument

in adder
44


In [19]:
print(logging_add39(5))

Calling <function make_adder.<locals>.adder at 0x7fb7e9791040>((5,), {})
in adder
44


In [20]:
logging_add39.__closure__[0].cell_contents

<function __main__.make_adder.<locals>.adder(y)>

## Decorators
* Wrapper functions are so common, that Python has its own term for it–a *decorator*.
* Why might you want to use a decorator?
  * sometimes you want to modify a function’s behavior without explicitly modifying the function, e.g., pre/post actions, debugging, etc. 
  * suppose we have a set of tasks that need to be performed by many different functions, e.g.,
   * access control
   * cleanup
   * error handling
   * logging
 * ...in other words, there is some boilerplate code that needs to be executed before or after  every invocation of the function


## Decorators build on topics we already know...
* nested functions
* variable positional args (`*args`)
* variable keyword args (`**kwargs`)
* functions are objects (actually everything in Python is an object)

In [ ]:
def document_it(func):
    # below is a nested, or inner function
    def new_function(*args, **kwargs):
        print(f'Running function: {func.__name__}')
        print(f'Positional arguments: {args}')
        print(f'Keyword arguments: {kwargs}')ßß
        # here we invoke the function passed in as an argument
        result = func(*args, **kwargs)
        print(f'Result: {result}')
        return result
    
    # document_it() is returning a reference to the inner function
    return new_function

In [ ]:
def add_things(a, b):
    return a + b

print('Running plain old add_things()')
print(add_things(13, 5))

In [ ]:
# manual decorator assignment
add_things = document_it(add_things) 

In [ ]:
#print('Running cooler_add_things()')
cooler_add_things(13, 5)

In [ ]:
# decorator shorthand for what we did above

# from your_decorators import document_it

@document_it
def add_things(a, b):
    return a + b

# add_things = document_it(add_things)

print(add_things(13, -5))

## Lab: Decorators
1. Create some function which takes an integer as its parameter
  * create a wrapper that ensures the parameter is positive
  * use that wrapper to decorate your original function
2. Make a timer decorator that computes the elapsed time of the function wrapped by it
